# Импорт библиотек

In [1]:
from selenium import webdriver
import time
from datetime import datetime
import re
import requests
import random
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Чтение файла

In [8]:
sellers_info = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\промежуточный итог18.06.xlsx")
sellers_info = sellers_info.iloc[2400:2409]#с 534 строки начинаются строки с оффициальными названиями, 2400-2409 - пробная выборка, чтобы быстро посчиталось
sellers_info

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0,Официальное название
2755,243,7727194489,https://www.wildberries.ru/seller/17467,ПАРФЮМЕРИЯ XXI ВЕКА,Парфюмерия,470 254,243.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ''ПАР...
2756,244,231136188300,https://www.wildberries.ru/seller/669142,SHAIKH PARFUM,Парфюмерия,93 668,244.0,Индивидуальный предприниматель Оруджова Шабнам...
2757,245,143410054042,https://www.wildberries.ru/seller/131239,ZAPAH,Парфюмерия,30 655,245.0,Индивидуальный предприниматель Немецких Наталь...
2758,246,9717169487,https://www.wildberries.ru/seller/4247314,Le’parfume,Парфюмерия,31 948,246.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""БАКАЙ"""
2759,247,581304781867,https://www.wildberries.ru/seller/183599,MARKETNA,Парфюмерия,22 365,247.0,Индивидуальный предприниматель Лебедева Натали...
2760,248,581304781867,https://www.wildberries.ru/seller/1035748,RENI-DUHI,Парфюмерия,?,248.0,Индивидуальный предприниматель Лебедева Натали...
2761,249,551405483106,https://www.wildberries.ru/seller/1420874,ROSE BREND,Парфюмерия,22 826,249.0,ИП
2762,250,9728076033,https://www.wildberries.ru/seller/1124756,Мир ароматов и подарков,Парфюмерия,69 166,250.0,"ООО""Миллиардер"""
2763,251,422320012901,https://www.wildberries.ru/seller/4489021,AMANI,Парфюмерия,547,251.0,Индивидуальный предприниматель А.А.
2764,252,770179803557,https://www.wildberries.ru/seller/4331607,Smellstar,Парфюмерия,3 062,252.0,Индивидуальный предприниматель Шарипов Комрон ...


# Основная функция парсинга

In [4]:
def find_Alice(official_name):
    options = webdriver.ChromeOptions()
    #options.add_argument(f"user-agent={random.choice(user_agent_list)}")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/"+ "537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
            
    # Запуск веб-драйвера
    #service = Service(executable_path=r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\Парсер яндекса\chromedriver.exe")
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 60)
            
    url = "https://yandex.ru/n"
    driver.get(url=url)
    element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "FuturisTextarea-Input")))
    time.sleep(0.2)

    search = official_name + ' контакты, выдели, пожалуйста только сам телефон или почту'
    input_place = driver.find_element(By.CLASS_NAME, 'FuturisTextarea-Input')#.format('Найдётся всё'))
    input_place.clear()
    input_place.send_keys(search)
    input_place.send_keys(Keys.ENTER)
    #element = WebDriverWait(driver, 10).until(
    #EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.Modal.Modal_visible.Modal_hasAnimation.Distribution-SplashScreenModal.SplashscreenTopButton.SplashscreenTopButton_layout_system_top_dark_close-button > div.Modal-Wrapper > div > div > div > div > div.Distribution-Actions > button")))
    #driver.find_element(By.CSS_SELECTOR, 'body > div.Modal.Modal_visible.Modal_hasAnimation.Distribution-SplashScreenModal.SplashscreenTopButton.SplashscreenTopButton_layout_system_top_dark_close-button > div.Modal-Wrapper > div > div > div > div > div.Distribution-Actions > button').click()
    try:
                        element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "FuturisGPTMessage-Disclaimer")))
                        time.sleep(0.2)
                        html_source = driver.find_element(By.CLASS_NAME, "FuturisMarkdown").get_attribute("outerHTML")
                        soup = BeautifulSoup(html_source, 'html.parser')
                        #print(soup)
                        #phone_number = soup.find('strong').text
                        #print(soup)
                        strong_tags = soup.find_all('strong')
                        result = ' '.join(tag.get_text() for tag in strong_tags)
    except:
                        print("didn' get anything from alice")
                        result = ''
    
    driver.close()
    driver.quit()
    print(result)
    return result

# Выделение телефонов и почт из ответов яндекса

In [9]:
import re

def extract_emails_and_phones(input_string):
    # Регулярное выражение для поиска email-адресов
    phone_pattern = r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(?\d{1,4}?\)?[-.\s]?)?\d{1,4}[-.\s]?\d{1,4}[-.\s]?\d{1,9}\b'
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

    
    # Регулярное выражение для поиска телефонных номеров
    #phone_pattern = r'\\b(?:\\+?\\d{1,3}[-.\\s]?)?(?:\\(?\\d{1,4}?\\)?[-.\\s]?)?\\d{1,4}[-.\\s]?\\d{1,4}[-.\\s]?\\d{1,9}\\b'
    
    # Поиск email-адресов
    emails = re.findall(email_pattern, input_string)
    
    # Поиск телефонных номеров
    phones = re.findall(phone_pattern, input_string)
    
    return emails, phones

# Пример использования
input_string = "Контактные данные: john.doe@example.com, 123-456-7890 и jane_doe123@gmail.com."
emails, phones = extract_emails_and_phones(input_string)
print("Электронные адреса:", emails)
print("Номера телефонов:", phones)

Электронные адреса: ['john.doe@example.com', 'jane_doe123@gmail.com']
Номера телефонов: ['123-456-7890']


# Применение к датасету

In [9]:
sellers_info['Телефон из Яндекса'] = sellers_info['Официальное название'].apply(find_Alice)
sellers_info['Контакты из Яндекса'] = sellers_info['Телефон из Яндекса'].apply(extract_emails_and_phones)
sellers_info

Телефон Электронная почта
soltanov_rasim@inbox.ru
контактные данные B2B.house VBankCenter.ru vypiska-nalog.com
ООО «БАКАЙ» ликвидировано
andriyanova_93@inbox.ru
andriyanova_93@inbox.ru
Проверить реестр ИП Использовать онлайн-сервисы Обратиться к социальным сетям Посетить официальные веб-сайты и публичные базы данных Обратиться к коллегам и партнёрам Воспользоваться услугами информационных агентств
телефон, электронная почта и сайт ООО «Миллиардер» не указаны
Онлайн-сервисы Социальные сети Официальные веб-сайты и публичные базы данных Сервисы для проверки организаций
komron.sharipov.90@bk.ru
телефон, электронная почта и другие контактные данные организации отсутствуют
sravni.ru vbankcenter.ru
ctilekov@mail.ru
Электронная почта teenka@yandex.ru
Электронная почта everest0505@mail.ru
Телефон Электронная почта


NameError: name 'extract_emails_and_phones' is not defined

# Сохранение промежуточного результата

In [25]:
sellers_info.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из яндекс нейро3.xlsx")
sellers_info

,Unnamed: 0.2,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0,Официальное название,Телефон из Яндекса,Телефон из Яндекса очищенные
0,534,0,7801479660,https://www.wildberries.ru/seller/93638,DeoGlory - модные аксессуары для нее и для него,Фэшен,13 960,0,КОНКОРД ООО,Телефоны Электронная почта,
1,535,1,710309019475,https://www.wildberries.ru/seller/343639,REYNO,Фэшен,23 392,1,0,RusTXT.ru bolshoyvopros.ru android.stackexchan...,
2,536,2,771893498655,https://www.wildberries.ru/seller/1296237,PENIMI,Фэшен,394 485,2,Индивидуальный предприниматель Пельтек Николай...,peltek78@mail.ru,"(['peltek78@mail.ru'], [])"
3,537,3,343706602704,https://www.wildberries.ru/seller/726131,BOYMEN,Фэшен,49 555,3,Индивидуальный предприниматель Суфиева Юлия Ол...,vash_buh80@mail.ru,"(['vash_buh80@mail.ru'], [])"
4,538,4,668509585739,https://www.wildberries.ru/seller/3972135,Belt.03,Фэшен,15 536,4,Индивидуальный предприниматель Азизходжаева Си...,телефон husenshakarov@gmail.com,"(['husenshakarov@gmail.com'], [])"
...,...,...,...,...,...,...,...,...,...,...,...
1871,2405,421,614045272974,https://www.wildberries.ru/seller/4392584,InspiriaHomme,Уходовая косметика,4 457,426,Индивидуальный предприниматель Князева Виктори...,Электронная почта,
1872,2406,422,781437789123,https://www.wildberries.ru/seller/789909,BeautyNation,Уходовая косметика,165 067,427,Индивидуальный предприниматель Костров Андрей ...,прекратил деятельность,
1873,2407,423,591793314686,https://www.wildberries.ru/seller/451152,Мужикова,Уходовая косметика,12 736,429,Индивидуальный предприниматель Мужикова Дарья ...,d.mujikova2012@yandex.ru,"(['d.mujikova2012@yandex.ru'], [])"
1874,2408,424,9701165487,https://www.wildberries.ru/seller/539261,BNS GROUP,Уходовая косметика,616 541,430,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""БНС ...",Контакты АО «БНС ГРУП» Телефон Электронная почта,


In [24]:
#sellers_info['Телефон из Яндекса очищенные'] = sellers_info['Телефон из Яндекса'].apply(extract_emails_and_phones)
sellers_info = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из яндекс нейро3.xlsx")
sellers_info.replace(to_replace='([], [])', value='', inplace=True)
sellers_info#['Телефон из Яндекса очищенные'].iloc[0]

,Unnamed: 0.2,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0,Официальное название,Телефон из Яндекса,Телефон из Яндекса очищенные
0,534,0,7801479660,https://www.wildberries.ru/seller/93638,DeoGlory - модные аксессуары для нее и для него,Фэшен,13 960,0,КОНКОРД ООО,Телефоны Электронная почта,
1,535,1,710309019475,https://www.wildberries.ru/seller/343639,REYNO,Фэшен,23 392,1,0,RusTXT.ru bolshoyvopros.ru android.stackexchan...,
2,536,2,771893498655,https://www.wildberries.ru/seller/1296237,PENIMI,Фэшен,394 485,2,Индивидуальный предприниматель Пельтек Николай...,peltek78@mail.ru,"(['peltek78@mail.ru'], [])"
3,537,3,343706602704,https://www.wildberries.ru/seller/726131,BOYMEN,Фэшен,49 555,3,Индивидуальный предприниматель Суфиева Юлия Ол...,vash_buh80@mail.ru,"(['vash_buh80@mail.ru'], [])"
4,538,4,668509585739,https://www.wildberries.ru/seller/3972135,Belt.03,Фэшен,15 536,4,Индивидуальный предприниматель Азизходжаева Си...,телефон husenshakarov@gmail.com,"(['husenshakarov@gmail.com'], [])"
...,...,...,...,...,...,...,...,...,...,...,...
1871,2405,421,614045272974,https://www.wildberries.ru/seller/4392584,InspiriaHomme,Уходовая косметика,4 457,426,Индивидуальный предприниматель Князева Виктори...,Электронная почта,
1872,2406,422,781437789123,https://www.wildberries.ru/seller/789909,BeautyNation,Уходовая косметика,165 067,427,Индивидуальный предприниматель Костров Андрей ...,прекратил деятельность,
1873,2407,423,591793314686,https://www.wildberries.ru/seller/451152,Мужикова,Уходовая косметика,12 736,429,Индивидуальный предприниматель Мужикова Дарья ...,d.mujikova2012@yandex.ru,"(['d.mujikova2012@yandex.ru'], [])"
1874,2408,424,9701165487,https://www.wildberries.ru/seller/539261,BNS GROUP,Уходовая косметика,616 541,430,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""БНС ...",Контакты АО «БНС ГРУП» Телефон Электронная почта,


In [42]:
df['Почты'] = df['Телефон из Яндекса очищенные'].apply(get_email)
df['Телефоны'] = df['Телефон из Яндекса очищенные'].apply(get_phone)
df['Телефоны с почтой'] = df['Телефон из Яндекса очищенные'].apply(get_email_and_phone)
df.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из яндекс нейро4.xlsx")
df

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Unnamed: 0,Официальное название,Телефон из Яндекса,Телефон из Яндекса очищенные,Почты,Телефоны,Телефоны с почтой
0,0,534,0,7801479660,https://www.wildberries.ru/seller/93638,DeoGlory - модные аксессуары для нее и для него,Фэшен,13 960,0,КОНКОРД ООО,Телефоны Электронная почта,NaN,,,
1,1,535,1,710309019475,https://www.wildberries.ru/seller/343639,REYNO,Фэшен,23 392,1,0,RusTXT.ru bolshoyvopros.ru android.stackexchan...,NaN,,,
2,2,536,2,771893498655,https://www.wildberries.ru/seller/1296237,PENIMI,Фэшен,394 485,2,Индивидуальный предприниматель Пельтек Николай...,peltek78@mail.ru,"(['peltek78@mail.ru'], [])",peltek78@mail.ru,,peltek78@mail.ru
3,3,537,3,343706602704,https://www.wildberries.ru/seller/726131,BOYMEN,Фэшен,49 555,3,Индивидуальный предприниматель Суфиева Юлия Ол...,vash_buh80@mail.ru,"(['vash_buh80@mail.ru'], [])",vash_buh80@mail.ru,,vash_buh80@mail.ru
4,4,538,4,668509585739,https://www.wildberries.ru/seller/3972135,Belt.03,Фэшен,15 536,4,Индивидуальный предприниматель Азизходжаева Си...,телефон husenshakarov@gmail.com,"(['husenshakarov@gmail.com'], [])",husenshakarov@gmail.com,,husenshakarov@gmail.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1871,1871,2405,421,614045272974,https://www.wildberries.ru/seller/4392584,InspiriaHomme,Уходовая косметика,4 457,426,Индивидуальный предприниматель Князева Виктори...,Электронная почта,NaN,,,
1872,1872,2406,422,781437789123,https://www.wildberries.ru/seller/789909,BeautyNation,Уходовая косметика,165 067,427,Индивидуальный предприниматель Костров Андрей ...,прекратил деятельность,NaN,,,
1873,1873,2407,423,591793314686,https://www.wildberries.ru/seller/451152,Мужикова,Уходовая косметика,12 736,429,Индивидуальный предприниматель Мужикова Дарья ...,d.mujikova2012@yandex.ru,"(['d.mujikova2012@yandex.ru'], [])",d.mujikova2012@yandex.ru,,d.mujikova2012@yandex.ru
1874,1874,2408,424,9701165487,https://www.wildberries.ru/seller/539261,BNS GROUP,Уходовая косметика,616 541,430,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""БНС ...",Контакты АО «БНС ГРУП» Телефон Электронная почта,NaN,,,


# Дальнейшая обработка и слияние с предыдущими данными

In [93]:
#df = df.drop(['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], axis=1)
df3 = df.drop(['Ссылка на карточку', 'Название', 'Фокусная отрасль(запрос)', 'Количество товаров продано', 'Официальное название', 'Телефон из Яндекса', 'Телефон из Яндекса очищенные', 'Телефоны с почтой', 'Телефоны'], axis=1)
#df3 = df3.dropna(subset = 'Телефоны с почтой')
df3.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из яндекс нейро6.xlsx")
df3 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из яндекс нейро6.xlsx")
df3 = df3.dropna(subset = 'Почты')
df3.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\из яндекс нейро6.xlsx")
df3

,Unnamed: 0,ИНН,Почты
2,2,771893498655,peltek78@mail.ru
3,3,343706602704,vash_buh80@mail.ru
4,4,668509585739,husenshakarov@gmail.com
5,5,668509988336,husenshakarov@gmail.com
6,6,667801869706,husenshakarov@gmail.com
...,...,...,...
1868,1868,236103920502,mrshelena@yandex.ru
1869,1869,263215424961,inga-am@mail.ru
1870,1870,770976024375,naser.nodirov@inbox.ru
1873,1873,591793314686,d.mujikova2012@yandex.ru


In [41]:
def get_email(string):
    return ", ".join(extract_emails_and_phones(str(string))[0])

def get_phone(string):
    return ", ".join(extract_emails_and_phones(str(string))[1])

def get_email_and_phone(string):
    email_and_phone =  ", ".join([", ".join(extract_emails_and_phones(str(string))[1]), ", ".join(extract_emails_and_phones(str(string))[0])])
    if email_and_phone == ", ":
        return ""
    elif email_and_phone[:2] == ', ':
        return email_and_phone[2:]
    else:
        return email_and_phone

In [54]:
df2 = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\собранное+из бд с почтой.xlsx")
#df2 = df2.drop_duplicates(subset = 'ИНН')
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
df2

,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Официальное название,region,status,Контакты
0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,Московская область,Действующее,CHUMAKOVA_VA@MAIL.RU
1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,Свердловская область,Действующее,MALLAEV.AMIR@BK.RU
2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,Архангельская область,Действующее,DMITRY_AKSENOV@HOTMAIL.COM
3,22301565032,https://www.wildberries.ru/seller/272815,SG fashion & accessory,Фэшен,361 529,NaN,NaN,NaN,NaN
4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,Москва,Действующее,SMART-HUNTER@YANDEX.RU
...,...,...,...,...,...,...,...,...,...
4168,070711860657,https://www.wildberries.ru/seller/1402440,Adam,Парфюмерия,23 834,Индивидуальный предприниматель Баркал Гада,NaN,NaN,NaN
4169,165064455557,https://www.wildberries.ru/seller/4248052,Парфюмные хиты,Парфюмерия,4 860,Индивидуальный предприниматель Тилеков Чынтемир,NaN,NaN,NaN
4170,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,Индивидуальный предприниматель Богословская Лю...,Москва,Действующее,TEENKA@YANDEX.RU
4171,350705516274,https://www.wildberries.ru/seller/4025176,Perfume Dreams,Парфюмерия,41,Индивидуальный предприниматель Атакишиева Айгю...,Вологодская область,Действующее,EVEREST0505@MAIL.RU


In [97]:
union = pd.merge(df2, df3, on = 'ИНН', how = 'left')#, suffixes=('_df1', '_df2'))
union['Контакты СПАРК и яндекс'] = union['Контакты'].fillna('') + ", " + union['Почты'].fillna("")
union['Контакты СПАРК и яндекс'] = union['Контакты СПАРК и яндекс'].apply(delete_m)
union.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\спарк+яндекс3.xlsx")
union = pd.read_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\спарк+яндекс3.xlsx")
union = union.dropna(subset = ['Контакты СПАРК и яндекс'])
union = union.drop_duplicates(subset = ['Контакты СПАРК и яндекс'])
#union = union.drop_duplicates(subset = ['ИНН'])
union = union.dropna(subset = ['ИНН'])
union.to_excel(r"\\pecom.local\UserData\WKS_FR_DCR\ermakov.dst\Desktop\Рабочая папка\WB\Седьмая партия\спарк+яндекс7.xlsx")
union

,Unnamed: 0.1,ИНН,Ссылка на карточку,Название,Фокусная отрасль(запрос),Количество товаров продано,Официальное название,region,status,Контакты,Unnamed: 0,Почты,Контакты СПАРК и яндекс
0,0,561507269404,https://www.wildberries.ru/seller/455568,MaryashA,Фэшен,149 268,NaN,Московская область,Действующее,CHUMAKOVA_VA@MAIL.RU,NaN,NaN,"CHUMAKOVA_VA@MAIL.RU,"
1,1,665821766737,https://www.wildberries.ru/seller/4234077,B-Store,Фэшен,70 652,NaN,Свердловская область,Действующее,MALLAEV.AMIR@BK.RU,NaN,NaN,"MALLAEV.AMIR@BK.RU,"
2,2,290209522869,https://www.wildberries.ru/seller/3945406,Brenda Fashion,Фэшен,8 964,NaN,Архангельская область,Действующее,DMITRY_AKSENOV@HOTMAIL.COM,NaN,NaN,"DMITRY_AKSENOV@HOTMAIL.COM,"
4,4,773001860640,https://www.wildberries.ru/seller/78504,B.B.Fashion,Фэшен,33 674,NaN,Москва,Действующее,SMART-HUNTER@YANDEX.RU,NaN,NaN,"SMART-HUNTER@YANDEX.RU,"
5,5,771577539871,https://www.wildberries.ru/seller/55034,XELA,Фэшен,507 689,NaN,Мурманская область,Действующее,COKOLOVCKI@GMAIL.COM,NaN,NaN,"COKOLOVCKI@GMAIL.COM,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4213,4213,231136188300,https://www.wildberries.ru/seller/669142,SHAIKH PARFUM,Парфюмерия,93 668,Индивидуальный предприниматель Оруджова Шабнам...,Волгоградская область,Действующее,SOLTANOV_RASIM@INBOX.RU,NaN,NaN,"SOLTANOV_RASIM@INBOX.RU,"
4216,4216,581304781867,https://www.wildberries.ru/seller/183599,MARKETNA,Парфюмерия,22 365,Индивидуальный предприниматель Лебедева Натали...,Московская область,Действующее,ANDRIYANOVA_93@INBOX.RU,NaN,NaN,"ANDRIYANOVA_93@INBOX.RU,"
4219,4219,551405483106,https://www.wildberries.ru/seller/1420874,ROSE BREND,Парфюмерия,22 826,ИП,Москва,Действующее,AINAGUL-78@MAIL.RU,NaN,NaN,"AINAGUL-78@MAIL.RU,"
4228,4228,622809640920,https://www.wildberries.ru/seller/624327,Арома Мэджик,Парфюмерия,77 844,Индивидуальный предприниматель Богословская Лю...,Москва,Действующее,TEENKA@YANDEX.RU,NaN,NaN,"TEENKA@YANDEX.RU,"


In [87]:
def delete_m(email_and_phone):
    if email_and_phone == " , ":
        return np.nan
    elif email_and_phone[:2] == ', ':
        return email_and_phone[2:]
    elif email_and_phone[-1:] == ',':
        return email_and_phone[:-1]
    else:
        return email_and_phone
    
    